In [ ]:
!pip install chromedriver-binary==84.0.4147.30.0

In [ ]:
!pip install selenium bs4 tqdm

In [ ]:
import os
import time
from pathlib import Path

import urllib
from urllib import request
import chromedriver_binary
from selenium import webdriver
from bs4 import BeautifulSoup
from tqdm import tqdm

In [ ]:
def get_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--no-sandbox")
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver


def scrape_art_wiki(driver):
    root_url = "https://www.wikiart.org"
    driver.get(root_url + "/en/paintings-by-style/")
    time.sleep(1.0)
    print(driver.title)
    bs = BeautifulSoup(driver.page_source)
    dict_list = bs.find("ul", class_="dictionaries-list")
    link_list = [li.a.get("href") for li in dict_list.find_all("li", class_="dottedItem")]

    style_image_dict = {}
    for link in link_list:
        style = link.split("/")[-1]
        style_image_dict[style] = scrape_art_wiki_page(driver, root_url + link)
    return style_image_dict


def scrape_art_wiki_page(driver, url):
    driver.get(url)
    while True:
        time.sleep(0.2)
        try:
            more_button = driver.find_element_by_class_name("masonry-load-more-button")
            more_button.click()
        except:
            break

    html = driver.page_source
    bs = BeautifulSoup(html, "html.parser")
    img_list = bs.find("ul", class_="wiki-masonry-container")
    img_urls = [img.get("src") for img in img_list.select("img")]
    img_urls = [img.split("!")[0] for img in img_urls if img.endswith("jpg")]
    print("Found {} images in {}".format(len(img_urls), url))
    return img_urls

In [ ]:
driver = get_driver()
style_image_dict = scrape_art_wiki(driver)

In [ ]:
def downloads(style_image_dict, save_root_dir):
    for key, paths in style_image_dict.items():
        style = key.split("?")[0]
        save_dir = os.path.join(save_root_dir, style)
        os.makedirs(save_dir, exist_ok=True)
        for path in tqdm(paths):
            download_impl(path, save_dir)
        print("style : {}".format(style))
            
def download_impl(url, save_dir):
    dst_path = os.path.join(save_dir, os.path.basename(url))
    if os.path.exists(dst_path):
        return
    try:
        with urllib.request.urlopen(url) as web_file:
            data = web_file.read()
            with open(dst_path, mode='wb') as local_file:
                local_file.write(data)
    except urllib.error.URLError as e:
        print(e)
    except:
        if os.path.exists(dst_path):
            os.remove(dst_path)

In [ ]:
save_root_dir = Path.home() / "dataset/ArtWiki"
downloads(style_image_dict, str(save_root_dir))

In [ ]:
len(style_image_dict)